In [1]:
source('cascade_helper.R')


### list of parameters and filenames

In [2]:
wd <- getwd()

p_thr <- 5e-9
maf_min_thr <- 1e-4

annot.tbl <- '/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz'

cascade_files <- file.path(wd, 'cascade.input.files.tsv')
out_array_hits <- file.path(wd, 'out_v3', 'cascade.array.hits.tsv')

gdrive_dir <- '1fGlnrj8Zu1Pox3fmxmFKdcbNVWiN9vhd'


In [3]:
traits <- fread(cascade_files)
dim(traits) %>% print()

[1] 35  4


#### read the variant annotation file for array

In [4]:
annot.arr <- read_annot_arr(annot.tbl)


#### read the GWAS summary statistics for the array

In [6]:
array_anno_unfiltered_df <- traits %>% 
read_array_sumstats_all(p_thr) %>%
annotate_array_df(annot.arr)


In [7]:
# quick check
array_anno_unfiltered_df %>% 
count(
    (as.numeric(P) < p_thr),
    is_autosome
)


(as.numeric(P) < p_thr),is_autosome,n
<lgl>,<lgl>,<int>
TRUE,TRUE,79390


In [8]:
array_anno_unfiltered_df %>% 
count(
    (as.numeric(maf) < maf_min_thr),    
    all_filters,
    is_outside_of_MHC,
    ld_indep
)


(as.numeric(maf) < maf_min_thr),all_filters,is_outside_of_MHC,ld_indep,n
<lgl>,<int>,<lgl>,<lgl>,<int>
FALSE,0,FALSE,FALSE,22356
FALSE,0,FALSE,TRUE,2676
FALSE,0,TRUE,FALSE,35143
FALSE,0,TRUE,TRUE,9735
FALSE,1,FALSE,FALSE,2457
FALSE,1,TRUE,FALSE,4720
FALSE,2,FALSE,FALSE,982
FALSE,2,TRUE,FALSE,1240
FALSE,3,FALSE,FALSE,12


In [9]:
array_anno_df <- array_anno_unfiltered_df %>%
filter(
    (as.numeric(P) < p_thr),
    is_autosome,
    (as.numeric(maf) > maf_min_thr),
    all_filters == 0,
    is_outside_of_MHC,
    ld_indep    
) %>%
select(
    -is_autosome,
    -filter,
    -hwe,
    -gnomad_af,
    -missingness,
    -mcpi
)


In [10]:
# quick check
array_anno_df %>% count(all_filters, is_outside_of_MHC, ld_indep)


all_filters,is_outside_of_MHC,ld_indep,n
<int>,<lgl>,<lgl>,<int>
0,TRUE,TRUE,9735


In [11]:
array_anno_df %>% 
select(Csq, is_rare, ID) %>%
unique() %>%
count(Csq, is_rare)


Csq,is_rare,n
<chr>,<lgl>,<int>
non-coding,FALSE,4389
non-coding,TRUE,75
protein-altering,FALSE,402
protein-altering,TRUE,192
protein-truncating,FALSE,10
protein-truncating,TRUE,28


In [12]:
array_anno_df %>%
fwrite(out_array_hits, sep='\t')


In [13]:
paste('gdrive', 'upload', '-p', gdrive_dir, out_array_hits, sep=' ')


[1] "gdrive upload -p 1fGlnrj8Zu1Pox3fmxmFKdcbNVWiN9vhd /oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/biomarkers/cascade/out_v3/cascade.array.hits.tsv"

In [15]:
paste('zstd', '--rm', '-15', out_array_hits, sep=' ')


[1] "zstd --rm -15 /oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/biomarkers/cascade/out_v3/cascade.array.hits.tsv"